In [ ]:
%load_ext watermark


In [ ]:
import itertools as it
import os

from IPython.display import display
import matplotlib as mpl
from matplotlib import container as mpl_container
import polars as pl
import seaborn as sns
from slugify import slugify
from teeplot import teeplot as tp


In [ ]:
%watermark -diwmuv -iv


In [ ]:
tp.save[".pgf"] = True
teeplot_subdir = os.environ.get(
    "NOTEBOOK_NAME", "2025-06-01-cpp-bench-memory-native-tilted"
)
teeplot_subdir


## Prep Data


In [ ]:
algo_names = {
    "dstream.circular_algo": "simple ringbuf",
    "control_throwaway_algo": "no-operation",
    "dstream.tilted_algo": "extended ringbuf",
    "dstream_tilted_algo": "extended ringbuf LUT",
    "doubling_tilted_algo": "naive doubling",
    "zhao_tilted_algo": "pyrimidal bucket",
    "zhao_tilted_full_algo": "saturating bucket",
}


df = pl.concat(
    [
        pl.read_csv("https://osf.io/cbqpx/download"),
    ],
)
print("\n".join(df["algo_name"].unique().to_list()))
df = (
    df.cast(
        {
            "memory_bytes": pl.Int32,
            "num_items": pl.Int32,
            "num_sites": pl.Int32,
            "duration_s": pl.Float64,
            "replicate": pl.Int32,
        },
    )
    .filter(
        pl.col("algo_name").is_in(
            algo_names.keys(),
        ),
        pl.col("num_items") == 1_000_000,
    )
    .with_columns(
        algorithm=pl.col("algo_name").map_elements(
            algo_names.__getitem__,
            return_dtype=str,
        ),
        num_retained=(
            pl.when(pl.col("algo_name") == "dstream_tilted_algo")
            .then(pl.col("num_sites"))
            .when(pl.col("algo_name") == "doubling_tilted_algo")
            .then(pl.col("num_sites") * 0.75)
            .when(pl.col("algo_name") == "zhao_tilted_algo")
            .then(19)
            .when(pl.col("algo_name") == "zhao_tilted_full_algo")
            .then(pl.col("num_sites"))
            .otherwise(pl.col("num_sites"))
        )
    )
    .with_columns(
        (pl.col("memory_bytes") / pl.col("num_retained")).alias("bytes per item"),
    )
    .with_columns(
        (8 * pl.col("memory_bytes") / pl.col("num_retained")).alias("bits per item"),
    )
    .with_columns(
        memory_bits=pl.col("memory_bytes") * 8,
        item_bits=(
            pl.col("data_type")
            .map_elements(
                {
                    "bit": 1, "byte": 8, "word": 16, "double word": 32
                }.get,
                return_dtype=pl.Int32,
            )
        ),
    )
    .with_columns(
        overhead_bits=(
            pl.col("memory_bits") - pl.col("item_bits") * pl.col("num_retained")
        ),
    )
    .with_columns(overhead_bytes=pl.col("overhead_bits") // 8)
    .with_columns(
        (
            100 * pl.col("item_bits") / pl.col("bits per item")
        ).alias("Memory Efficiency (%)"),
    )
    .with_columns(pl.col("data_type").alias("data type"))
)

display(df.describe()), display(df.head()), display(df.tail());


In [ ]:
assert (df.group_by(
    ["num_items", "num_sites", "algorithm", "data type"],
).agg(
    pl.col("memory_bytes").n_unique(),
)["memory_bytes"] == 1).all()


In [ ]:
hue_order = [
    "extended ringbuf",
    "saturating bucket",
    "pyrimidal bucket",
    # "extended ringbuf LUT",
    # "simple ringbuf",
    "naive doubling",
]


In [ ]:
palette = sns.color_palette("muted")
palette_assignments = {
    "extended ringbuf": palette[0],
    "extended ringbuf LUT": palette[1],
    "saturating bucket": palette[2],
    "pyrimidal bucket": palette[3],
    "simple ringbuf": palette[4],
    "naive doubling": palette[5],
    "no-operation": palette[6],
}


## Plot


## Memory Per Item --- Stacked


In [ ]:
for unit, legend, excl, rc in it.product(
    ["bits", "bytes"],
    [True, False],
    [[], ["pyrimidal bucket"]],
    [{}, {"font.family": "serif"}],
):
    excl_hue_order = [hue for hue in hue_order if hue not in excl]
    with mpl.rc_context(rc=rc):
        with tp.teed(
            sns.catplot,
            data=df.filter(
                pl.col("num_items") == 1_000_000,
            ).with_columns(
                stat=pl.lit(f"Memory per Item ({unit})\n"),
            ),
            x="num_sites",
            y=f"{unit} per item",
            hue="algorithm",
            hue_order=excl_hue_order,
            col="stat",
            row="data type",
            row_order=["bit", "byte", "word", "double word"],
            kind="bar",
            errorbar=None,
            margin_titles=True,
            legend=legend,
            aspect=1.7,
            height=1.6,
            palette=[*map(palette_assignments.get, excl_hue_order)],
            sharex=True,
            sharey=False,
            teeplot_outattrs={
                "excl": slugify("-".join(excl)),
                "legend": legend,
                **rc,
            },
            teeplot_subdir=teeplot_subdir,
        ) as g:
            g.set(ylim=(0, None), yticks=[])
            g.set_titles(
                col_template="{col_name}",
                row_template="dtype\n{row_name}"
            )
            g.set_xlabels("Buffer Capacity\n(item count)")
            g.set_ylabels(f"")
            if legend:
                sns.move_legend(
                    g,
                    "lower center",
                    bbox_to_anchor=(0.37, 0.98),
                    columnspacing=0.7,
                    labelspacing=0.2,
                    ncol=3,
                    frameon=False,
                    title=None,
                )
            for idx, ax in enumerate(g.axes.flat):
                ax.set_ylim(0, ax.get_ylim()[1] * 1.5)
                for container in ax.containers:
                    anns = ax.bar_label(
                        container,
                        fmt=" %.1f",
                        label_type="edge",
                        rotation=90,
                        padding=2,
                    )
                    for ann in anns:
                        ann_x, ann_y = ann.get_position()
                        ann.set_position((ann_x + 1.35, ann_y))

                unit_scale = {"bits": 1, "bytes": 1 / 8}[unit]
                bits = [1, 8, 16, 32][idx]
                target = bits * unit_scale
                ax.axhline(
                    target,
                    color="k",
                    linestyle="--",
                )
                ax.set_yticks([bits * unit_scale])
                if str(target) in ("1", "1.0"):
                    unit_ = unit[:-1]  # drop s
                else:
                    unit_ = unit
                ax.set_yticklabels(
                    [f"{target} {unit_}"],
                    rotation=90,
                    rotation_mode="anchor",
                    ha="left",
                )
            g.tight_layout()
            g.figure.subplots_adjust(wspace=0.05)


## Memory Per Item


In [ ]:
for unit, mark, excl, rc in it.product(
    ["bits", "bytes"],
    [True, False],
    [[], ["pyrimidal bucket"]],
    [{}, {"font.family": "serif"}],
):
    excl_hue_order = [hue for hue in hue_order if hue not in excl]
    with mpl.rc_context(rc=rc):
        with tp.teed(
            sns.catplot,
            data=df.filter(
                pl.col("num_items") == 1_000_000,
            ),
            x="num_sites",
            y=f"{unit} per item",
            hue="algorithm",
            hue_order=excl_hue_order,
            col="data type",
            col_wrap=2,
            col_order=["bit", "byte", "word", "double word"],
            kind="bar",
            errorbar=None,
            margin_titles=True,
            aspect=1.4,
            height=1.6,
            palette=[*map(palette_assignments.get, excl_hue_order)],
            sharex=True,
            sharey=False,
            teeplot_outattrs={
                "excl": slugify("-".join(excl)),
                "mark": mark,
                **rc,
            },
            teeplot_subdir=teeplot_subdir,
        ) as g:
            g.set(ylim=(0, None), yticks=[])
            g.set_titles(col_template="{col_name} dtype")
            g.set_xlabels("Buffer Capacity\n(item count)")
            g.set_ylabels(f"Memory per\nItem ({unit})")
            sns.move_legend(
                g,
                "lower center",
                bbox_to_anchor=(0.37, 0.95),
                columnspacing=0.7,
                labelspacing=0.2,
                ncol=2,
                frameon=False,
                title=None,
            )
            for idx, ax in enumerate(g.axes.flat):
                ax.set_ylim(0, ax.get_ylim()[1] * 1.75)
                for container in ax.containers:
                    anns = ax.bar_label(
                        container,
                        fmt=" %.1f",
                        label_type="edge",
                        rotation=90,
                        padding=2,
                    )
                    for ann in anns:
                        ann_x, ann_y = ann.get_position()
                        ann.set_position((ann_x + 1.35, ann_y))

                if mark:
                    unit_scale = {"bits": 1, "bytes": 1 / 8}[unit]
                    bits = [1, 8, 16, 32][idx]
                    ax.axhline(
                        bits * unit_scale,
                        color="k",
                        linestyle="--",
                    )
            g.tight_layout()
            g.figure.subplots_adjust(wspace=0.05)


## Memory Efficiency


In [ ]:
for mark, excl, rc in it.product(
    [True, False],
    [[], ["pyrimidal bucket"]],
    [{}, {"font.family": "serif"}],
):
    excl_hue_order = [hue for hue in hue_order if hue not in excl]
    with mpl.rc_context(rc=rc):
        with tp.teed(
            sns.catplot,
            data=df.filter(
                pl.col("num_items") == 1_000_000,
            ),
            x="num_sites",
            y="Memory Efficiency (%)",
            hue="algorithm",
            hue_order=excl_hue_order,
            col="data type",
            col_wrap=2,
            col_order=["bit", "byte", "word", "double word"],
            kind="bar",
            errorbar=None,
            margin_titles=True,
            aspect=1.5,
            height=1.5,
            gap=0.0,
            palette=[*map(palette_assignments.get, excl_hue_order)],
            sharex=True,
            sharey=True,
            teeplot_outattrs={
                "excl": slugify("-".join(excl)),
                "mark": mark,
                **rc,
            },
            teeplot_subdir=teeplot_subdir,
        ) as g:
            g.set(ylim=(0, None))
            g.set_titles(col_template="{col_name} dtype")
            g.set_xlabels("Buffer Capacity\n(item count)")
            g.set_ylabels("Memory\nEfficiency (%)")
            sns.move_legend(
                g,
                "lower center",
                bbox_to_anchor=(0.4, 0.95),
                columnspacing=0.7,
                labelspacing=0.2,
                ncol=2,
                frameon=False,
                title=None,
            )
            for idx, ax in enumerate(g.axes.flat):
                ax.set_ylim(0, 100)
                for container in ax.containers:
                    for bar in container:
                        shrink = bool(bar.get_height() < 50)
                        ann = ax.bar_label(
                            mpl_container.BarContainer(
                                [bar],
                                datavalues=[bar.get_height()],
                                orientation="vertical",
                            ),
                            color=["white", "black"][shrink],
                            fmt=" %.0f%%",
                            fontsize=7.5,
                            label_type=["center", "edge"][shrink],
                            rotation=90,
                        )[0]
                        ann_x, ann_y = ann.get_position()
                        ann.set_position((ann_x + 1.25, ann_y))

                if mark:
                    ax.axhline(
                        100,
                        color="k",
                        linestyle="--",
                    )
            g.tight_layout()


## Memory Efficiency --- Stacked


In [ ]:
for legend, excl, rc in it.product(
    [True, False],
    [[], ["pyrimidal bucket"]],
    [{}, {"font.family": "serif"}],
):
    excl_hue_order = [hue for hue in hue_order if hue not in excl]
    with mpl.rc_context(rc=rc):
        with tp.teed(
            sns.catplot,
            data=df.filter(
                pl.col("num_items") == 1_000_000,
            ).with_columns(
                stat=pl.lit("Memory Efficiency (%)\n"),
            ),
            x="num_sites",
            y="Memory Efficiency (%)",
            hue="algorithm",
            hue_order=excl_hue_order,
            col="stat",
            row="data type",
            row_order=["bit", "byte", "word", "double word"],
            kind="bar",
            errorbar=None,
            legend=legend,
            margin_titles=True,
            aspect=1.7,
            height=1.6,
            gap=0.0,
            palette=[*map(palette_assignments.get, excl_hue_order)],
            sharex=True,
            sharey=True,
            teeplot_outattrs={
                "excl": slugify("-".join(excl)),
                "legend": legend,
                **rc,
            },
            teeplot_subdir=teeplot_subdir,
        ) as g:
            g.set(ylim=(0, None))
            g.set_titles(
                col_template="{col_name}",
                row_template="dtype\n{row_name}"
            )
            g.set_xlabels("Buffer Capacity\n(item count)")
            g.set_ylabels("")
            if legend:
                sns.move_legend(
                    g,
                    "lower center",
                    bbox_to_anchor=(0.4, 0.98),
                    columnspacing=0.7,
                    labelspacing=0.2,
                    ncol=2,
                    frameon=False,
                    title=None,
                )
            for idx, ax in enumerate(g.axes.flat):
                ax.set_ylim(0, 100)
                ax.set_yticks([100])
                ax.set_yticklabels([" "])
                for container in ax.containers:
                    for bar in container:
                        shrink = bool(bar.get_height() < 50)
                        ann = ax.bar_label(
                            mpl_container.BarContainer(
                                [bar],
                                datavalues=[bar.get_height()],
                                orientation="vertical",
                            ),
                            color=["white", "black"][shrink],
                            fmt=" %.0f%%",
                            fontsize=7.5,
                            label_type=["center", "edge"][shrink],
                            rotation=90,
                        )[0]
                        ann_x, ann_y = ann.get_position()
                        ann.set_position((ann_x + 1.25, ann_y))

                ax.axhline(
                    100,
                    color="k",
                    linestyle="--",
                )
                ax.annotate(
                    "100%",
                    xy=(1.0, 1.0),
                    xycoords="axes fraction",
                    ha="left",
                    va="center",
                    size="small",
                    clip_on=False,
                )
            g.tight_layout()


## Memory Footprint


In [ ]:
for unit, mark, excl, rc in it.product(
    ["bytes"],
    [True, False],
    [[], ["pyrimidal bucket"]],
    [{}, {"font.family": "serif"}],
):
    excl_hue_order = [hue for hue in hue_order if hue not in excl]
    with mpl.rc_context(rc=rc):
        with tp.teed(
            sns.catplot,
            data=df.filter(
                pl.col("num_items") == 1_000_000,
            ),
            y=f"memory_{unit}",
            hue="algorithm",
            hue_order=excl_hue_order,
            col="data type",
            row="num_sites",
            col_order=["bit", "byte", "word", "double word"],
            kind="bar",
            errorbar=None,
            margin_titles=True,
            aspect=1.8,
            height=1.3,
            palette=[*map(palette_assignments.get, excl_hue_order)],
            sharex=True,
            sharey=False,
            teeplot_outattrs={
                "excl": slugify("-".join(excl)),
                "mark": mark,
                **rc,
            },
            teeplot_subdir=teeplot_subdir,
        ) as g:
            g.set(ylim=(0, None), xticks=[], yticks=[])
            g.set_titles(
                col_template="{col_name} dtype",
                row_template="Buffer\nCapacity\n{row_name} items",
            )
            g.set_ylabels(f"Memory\nFootprint\n({unit})")
            sns.move_legend(
                g,
                "lower center",
                bbox_to_anchor=(0.4, 0.98),
                columnspacing=0.7,
                labelspacing=0.2,
                ncol=4,
                frameon=False,
                title=None,
            )
            for idx, ax in enumerate(g.axes.flat):
                ax.set_ylim(0, ax.get_ylim()[1] * 1.8)
                ax.tick_params(axis="y", labelrotation=60)
                for container in ax.containers:
                    ax.bar_label(
                        container,
                        fmt=" %.0f",
                        label_type="edge",
                        rotation=90,
                        padding=2,
                    )
                if mark:
                    unit_scale = {"bits": 1, "bytes": 1 / 8}[unit]
                    bits = [1, 8, 16, 32][idx % 4]
                    num_sites = [64, 256, 1024, 4096][idx // 4]
                    ax.axhline(
                        bits * unit_scale * num_sites,
                        color="k",
                        linestyle="--",
                    )
            g.tight_layout()


## Memory Overhead


In [ ]:
for unit, mark, excl, rc in it.product(
    ["bytes"],
    [True, False],
    [[], ["pyrimidal bucket"]],
    [{}, {"font.family": "serif"}],
):
    excl_hue_order = [hue for hue in hue_order if hue not in excl]
    with mpl.rc_context(rc=rc):
        with tp.teed(
            sns.catplot,
            data=df.filter(
                pl.col("num_items") == 1_000_000,
            ),
            x="num_sites",
            y=f"overhead_{unit}",
            hue="algorithm",
            hue_order=excl_hue_order,
            col="data type",
            col_wrap=2,
            col_order=["bit", "byte", "word", "double word"],
            kind="bar",
            errorbar=None,
            margin_titles=True,
            aspect=2,
            height=1.6,
            palette=[*map(palette_assignments.get, excl_hue_order)],
            sharex=True,
            sharey=True,
            teeplot_outattrs={
                "excl": slugify("-".join(excl)),
                "mark": mark,
                **rc,
            },
            teeplot_subdir=teeplot_subdir,
        ) as g:
            g.set(ylim=(0, None), yticks=[])
            g.set_titles(col_template="{col_name} dtype")
            g.set_xlabels("Buffer Capacity (item count)")
            g.set_ylabels(f"Memory\nOverhead\n({unit})")
            sns.move_legend(
                g,
                "lower center",
                bbox_to_anchor=(0.4, 0.95),
                columnspacing=0.7,
                labelspacing=0.2,
                ncol=4,
                frameon=False,
                title=None,
            )
            for idx, ax in enumerate(g.axes.flat):
                ax.set_ylim(0, ax.get_ylim()[1] * 1.6)
                for container in ax.containers:
                    ax.bar_label(
                        container,
                        fmt=" %.0f",
                        label_type="edge",
                        rotation=90,
                        padding=2,
                    )
                if mark:
                    unit_scale = {"bits": 1, "bytes": 1 / 8}[unit]
                    bits = [1, 8, 16, 32][idx]
                    ax.axhline(
                        bits * unit_scale,
                        color="k",
                        linestyle="--",
                    )
